# Task 4.8

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create path
path = r'C:\Users\RebekkaBrandt\Documents\Instacart Basket Analysis'


In [3]:
# Import orders_products_merged.pkl
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_2.pkl'))

In [4]:
# mean of order number by department
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})


,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


Creating the mean for the entire data set has shown all the departments which might change what we consider high / low volume orders

## Creating Loyalty Flag

In [5]:
# group and assign max value for oder number
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_previous_order,product_id,add_to_cart_order,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_peripd_of_day,busiest_period_of_day,max_order
0,0,0,2539329,1,1,2,8,NaN,196,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,Average Orders,Average Orders,10
1,1,1,2398795,1,2,3,7,15.0,196,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average Orders,Average Orders,10
2,2,2,473747,1,3,3,12,21.0,196,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most Orders,Most Orders,10
3,3,3,2254736,1,4,4,7,29.0,196,1,...,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Average Orders,Average Orders,10
4,4,4,431534,1,5,4,15,28.0,196,1,...,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Most Orders,Most Orders,10


In [7]:
# creating loyalts flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [9]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [12]:
# create mean, min, max prices for loyalty flag
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

It seems like new customers spend more on average than loyal customers

## Create Spending flag for all Users

In [13]:
# average price for each user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [14]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_previous_order,product_id,add_to_cart_order,...,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_peripd_of_day,busiest_period_of_day,max_order,loyalty_flag,avg_price
0,0,0,2539329,1,1,2,8,NaN,196,1,...,9.0,both,Mid-range product,Regularly busy,Regularly busy days,Average Orders,Average Orders,10,New customer,6.367797
1,1,1,2398795,1,2,3,7,15.0,196,1,...,9.0,both,Mid-range product,Regularly busy,Least busy days,Average Orders,Average Orders,10,New customer,6.367797
2,2,2,473747,1,3,3,12,21.0,196,1,...,9.0,both,Mid-range product,Regularly busy,Least busy days,Most Orders,Most Orders,10,New customer,6.367797
3,3,3,2254736,1,4,4,7,29.0,196,1,...,9.0,both,Mid-range product,Least busy,Least busy days,Average Orders,Average Orders,10,New customer,6.367797
4,4,4,431534,1,5,4,15,28.0,196,1,...,9.0,both,Mid-range product,Least busy,Least busy days,Most Orders,Most Orders,10,New customer,6.367797


In [15]:
# creating Spender flag
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low Spender'

In [16]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High Spender'

In [17]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_previous_order,product_id,add_to_cart_order,...,_merge,price_range_loc,busiest_day,busiest_days,busiest_peripd_of_day,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag
0,0,0,2539329,1,1,2,8,NaN,196,1,...,both,Mid-range product,Regularly busy,Regularly busy days,Average Orders,Average Orders,10,New customer,6.367797,Low Spender
1,1,1,2398795,1,2,3,7,15.0,196,1,...,both,Mid-range product,Regularly busy,Least busy days,Average Orders,Average Orders,10,New customer,6.367797,Low Spender
2,2,2,473747,1,3,3,12,21.0,196,1,...,both,Mid-range product,Regularly busy,Least busy days,Most Orders,Most Orders,10,New customer,6.367797,Low Spender
3,3,3,2254736,1,4,4,7,29.0,196,1,...,both,Mid-range product,Least busy,Least busy days,Average Orders,Average Orders,10,New customer,6.367797,Low Spender
4,4,4,431534,1,5,4,15,28.0,196,1,...,both,Mid-range product,Least busy,Least busy days,Most Orders,Most Orders,10,New customer,6.367797,Low Spender


## Create order frequency flag

In [18]:
# average price for each user
ords_prods_merge['order_freq'] = ords_prods_merge.groupby(['user_id'])['days_since_previous_order'].transform(np.median)

In [19]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_previous_order,product_id,add_to_cart_order,...,price_range_loc,busiest_day,busiest_days,busiest_peripd_of_day,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,order_freq
0,0,0,2539329,1,1,2,8,NaN,196,1,...,Mid-range product,Regularly busy,Regularly busy days,Average Orders,Average Orders,10,New customer,6.367797,Low Spender,20.5
1,1,1,2398795,1,2,3,7,15.0,196,1,...,Mid-range product,Regularly busy,Least busy days,Average Orders,Average Orders,10,New customer,6.367797,Low Spender,20.5
2,2,2,473747,1,3,3,12,21.0,196,1,...,Mid-range product,Regularly busy,Least busy days,Most Orders,Most Orders,10,New customer,6.367797,Low Spender,20.5
3,3,3,2254736,1,4,4,7,29.0,196,1,...,Mid-range product,Least busy,Least busy days,Average Orders,Average Orders,10,New customer,6.367797,Low Spender,20.5
4,4,4,431534,1,5,4,15,28.0,196,1,...,Mid-range product,Least busy,Least busy days,Most Orders,Most Orders,10,New customer,6.367797,Low Spender,20.5


In [29]:
# creating frequency flag
ords_prods_merge.loc[ords_prods_merge['order_freq'] >20, 'frequency_flag'] = 'Non-frequent customer'

In [34]:
ords_prods_merge.loc[(ords_prods_merge['order_freq'] >10) & (ords_prods_merge['order_freq'] <= 20), 'frequency_flag'] = 'Regular customer'

In [31]:
ords_prods_merge.loc[ords_prods_merge['order_freq'] <=10, 'frequency_flag'] = 'Frequent customer'

In [32]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_previous_order,product_id,add_to_cart_order,...,busiest_days,busiest_peripd_of_day,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,order_freq,frequency,frequency_flag
0,0,0,2539329,1,1,2,8,NaN,196,1,...,Regularly busy days,Average Orders,Average Orders,10,New customer,6.367797,Low Spender,20.5,NaN,Non-frequent customer
1,1,1,2398795,1,2,3,7,15.0,196,1,...,Least busy days,Average Orders,Average Orders,10,New customer,6.367797,Low Spender,20.5,NaN,Non-frequent customer
2,2,2,473747,1,3,3,12,21.0,196,1,...,Least busy days,Most Orders,Most Orders,10,New customer,6.367797,Low Spender,20.5,NaN,Non-frequent customer
3,3,3,2254736,1,4,4,7,29.0,196,1,...,Least busy days,Average Orders,Average Orders,10,New customer,6.367797,Low Spender,20.5,NaN,Non-frequent customer
4,4,4,431534,1,5,4,15,28.0,196,1,...,Least busy days,Most Orders,Most Orders,10,New customer,6.367797,Low Spender,20.5,NaN,Non-frequent customer


In [35]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [36]:
#export data
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_3.pkl'))